# libraries

In [ ]:
import numpy as np
import pandas as pd
from random import choice
import time
from tqdm import trange
import itertools
from itertools import combinations
from multiprocessing import Pool
import multiprocessing

# Reading Data and store it

In [ ]:
Tags = pd.read_csv("SOF Dataset/tags.txt" , sep="	" )
Tags = Tags.rename(columns={'Id': 'QId'})
Questions = pd.read_csv(
    "SOF Dataset/Questions.txt", encoding="UTF-16 LE", keep_default_na=False
)
Questions = Questions.sort_values(["Score", "ViewCount"], ascending=(False, False))
Questions.head()

,Id,CreationDate,Score,ViewCount,OwnerUserId,CommentCount,ClosedDate,FavoriteCount
253548,11227809,2012-06-27 13:51:36,9802,556194,87234,30,,5418
138111,6841333,2011-07-27 08:15:58,3072,288013,342235,16,,884
185096,8710619,2012-01-03 10:10:32,1852,107055,1127571,8,,436
256,40480,2008-09-02 20:14:29,1832,587017,4315,14,,974
2775,271526,2008-11-07 08:31:40,1728,524178,34856,26,,835


In [ ]:
def get_related_tags (Tags,inp_tag):
    l = Tags.loc[Tags['Tags']==inp_tag]['QId'].to_list()
    link = []
    link = Tags.loc[Tags['QId'].isin(l) , 'Tags'].tolist()
    return (set(link))

In [ ]:
def similarity(Tags,inp_tags):
    rel_tags_1 = get_related_tags (Tags,inp_tags[0])
    rel_tags_2 = get_related_tags (Tags,inp_tags[1])
    similarity = [sim for sim in rel_tags_1 if sim in rel_tags_2]
    return len(similarity) / (len(rel_tags_2) + len(rel_tags_1))

In [ ]:
def find_tags(dataset,s_range,e_range):
    dataset['Tags'].value_counts()
    countdt = pd.DataFrame(dataset['Tags'].value_counts()).rename_axis('unique_values').reset_index()
    countdt.columns = ['unique_values','count']
    return countdt[(countdt['count'] >= s_range) & (countdt['count'] < e_range)]

In [ ]:
def findTop(tags_comb):
    a = []
    for i in range(len(tags_comb)):
        a.append((tags_comb[i],similarity(Tags,[tags_comb[i][0] , tags_comb[i][1]])))
    return a

In [ ]:
Top_picks_ById = Questions[:1000]['Id'].tolist()
Tags = Tags.groupby('Tags').filter(lambda x : len(x)>1)
Top_picks_ByTag = Tags.loc[Tags['QId'].isin(Top_picks_ById) , 'Tags']
Top_picks_ByTag = pd.DataFrame(Top_picks_ByTag)
Top_picks_ByTag

,Tags
167,memory
910,jpa
2841,string
3825,android
4439,eclipse
...,...
1852771,initialization
1853252,xml
1853886,date
1854512,http


In [ ]:
totalTags = find_tags(Top_picks_ByTag,2,67)['unique_values'].tolist()

In [ ]:
tags_comb = list(set(combinations(totalTags, 2)))
len(tags_comb)

49141

In [ ]:
pool = multiprocessing.Pool(8)
tops = pool.map(findTop,[tags_comb])

In [ ]:
top10 = tops[0]

In [ ]:
top10.sort(key=lambda tup: tup[1], reverse=True)
top10[:10]

[(('hibernate', 'jpa'), 0.31517128874388256),
 (('servlets', 'jsp'), 0.30027207519168936),
 (('pass-by-reference', 'pass-by-value'), 0.2985781990521327),
 (('junit', 'unit-testing'), 0.29425182481751827),
 (('spring', 'spring-mvc'), 0.2936039650696247),
 (('jdbc', 'mysql'), 0.28785607196401797),
 (('loops', 'for-loop'), 0.2868080094228504),
 (('spring', 'java-ee'), 0.28620689655172415),
 (('arrays', 'arraylist'), 0.28309979982842437),
 (('character-encoding', 'utf-8'), 0.28152753108348133)]